In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
import sklearn.metrics as metrics
from dm_test.dm_test import dm_test as dm

In [2]:
with open('results_pkl/proposed_dynamic_n.pkl', 'rb') as handle:
    dict_results_dyn = pkl.load(handle)
    
with open('results_pkl/proposed.pkl', 'rb') as handle:
    dict_results = pkl.load(handle)


value_res = dict_results['value']
value_res_dyn = dict_results_dyn['value']

In [3]:
def get_percentage_difference(base, proposed):
    return ((base-proposed)/base)*100

model_columns = ["Static"]
df_percentage_mse = pd.DataFrame(columns = ["Time Series"] + model_columns)


for file_name, file_res in value_res.items():
    new_row = {'Time Series': file_name}
    real = file_res['Real']
    proposed_static = file_res['Proposed']
    mse_proposed_static = metrics.mean_squared_error(real, proposed_static)    
    proposed_dynamic = value_res_dyn[file_name]['Proposed']
    mse_proposed_dynamic = metrics.mean_squared_error(real, proposed_dynamic)   
    new_row['Static'] = get_percentage_difference(mse_proposed_static, mse_proposed_dynamic)
    new_row = pd.Series(new_row)
    df_percentage_mse = pd.concat([df_percentage_mse, new_row.to_frame().T], ignore_index=True)
    
        
df_percentage_mse[model_columns] = df_percentage_mse[model_columns].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df_percentage_mse.loc['Average'] = df_percentage_mse.mean(numeric_only=True)
df_percentage_mse.to_csv('results/dynamic/percentage_difference_mse.csv', index=False, float_format="%.2f")